In [1]:
import numpy as np

# 5.2 連鎖律
- 合成関数の微分の性質
- 高速に勾配を計算する誤差逆伝播法に応用する
- ニューラルネット全体の関数を，各ノードの局所的な関数とその先の様々な関数の合成関数と考えて応用

$$
z = f(t) \\
t = g(x, y)
$$
とすると，次のように合成関数の微分を求めることができる
$$
\frac{\partial z}{\partial x} = \frac{\partial z}{\partial t} \frac{\partial t}{\partial x}
$$

***

### 単純な乗算レイヤ，加算レイヤ

- 単純な乗算レイヤの例．
- 様々な複雑な演算を含む損失関数を $ L(z) $ とする．
- その内，一つの乗算ノードで $ z = xy $ という演算が行われるとする．

このとき，次のように勾配を求めることができる
$$
\frac{\partial L}{\partial x} = \frac{\partial L}{\partial z} \frac{\partial z}{\partial x} = \frac{\partial L}{\partial z} y \\　\\
\frac{\partial L}{\partial y} = \frac{\partial L}{\partial z} \frac{\partial z}{\partial y} = \frac{\partial L}{\partial z} x
$$
- $ \frac{\partial L}{\partial z} $ は乗算ノードの出力 $ z $ での $ L $ の偏微分．
- $ x, y $ を勾配計算に用いるため，順伝播の入力を保持しておく必要がある．

In [2]:
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None
        
    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x * y
        return out
    
    def backward(self, dout):
        dx = dout * self.y
        dy = dout * self.x
        return dx, dy

In [3]:
class AddLayer:
    def __init__(self):
        self.x = None
        self.y = None
        
    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x + y
        
        return out
    
    def backward(self, dout):
        dx = dout
        dy = dout
        
        return dx, dy

# 5.5 活性化関数レイヤ

### ReLUレイヤ
#### ReLU
$$
y = 
\begin{cases}
    x \quad (x \gt 0) \\
    0 \quad (x \leq 0)
\end{cases}
$$

#### ReLUの偏微分
$$
\frac{\partial y}{\partial x} = 
\begin{cases}
    1 \quad (x \gt 0) \\
    0 \quad (x \leq 0)
\end{cases}
$$

In [4]:
class Relu:
    def __init__(self):
        self.mask = None
        
    def forward(self, x):
        # 入力が 0 以下のインデックスを保持し，その値は0とする
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0
        
        return out
    
    def backward(self, dout):
        # 順伝播の入力が 0 以下であったインデックスの値は 0 ，そのほかはそのまま逆伝播
        dout[self.mask] = 0
        dx = dout
        
        return dx

### Sigmoidレイヤ
#### Sigmoid
$$
y = \frac{1}{1 + \exp(-x)}
$$

#### Sigmoidの偏微分導出
$$
t = 1 + \exp(-x)
$$

とおくと，
$$
y = \frac{1}{1 + \exp(-x)} = \frac{1}{t} = t^{-1}
$$

また，
$$
\begin{align}
\frac{\partial y}{\partial t} &= -t^{-2} = -(1 + \exp(-x))^{-2}\\
\frac{\partial t}{\partial x} &= -\exp(-x)
\end{align}
$$

であるから，
$$
\begin{align}
\frac{\partial y}{\partial x} &= \frac{\partial y}{\partial t} \frac{\partial t}{\partial x} \\ \\
&= \frac{\exp(-x)}{\{1 + \exp(-x)\}^{2}} \\ \\
&= \frac{\exp(-x)}{1 + \exp(-x)} \frac{1}{1 + \exp(-x)} \\ \\
&= \left(\frac{1 + \exp(-x)}{1 + \exp(-x)} - \frac{1}{1 + \exp(-x)} \right) \frac{1}{1 + \exp(-x)} \\ \\
&= (1 - y) y
\end{align}
$$

In [5]:
class Sigmoid:
    def __init__(self):
        self.out = None
        
    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out
        
        return out
    
    def backward(self, dout):
        # 逆伝播の入力に，Sigmoidの偏微分をかける
        dx = dout * (1.0 - self.out) * self.out
        
        return dx

# 5.6 Affine / Softmaxレイヤ

### Affine変換
$$
Y = X \cdot W + B
$$

- $ X, W, B $ は多次元配列
- 順伝播における，入力 $ X $，重み $ W $，バイアス $ B $ を表す
- 損失関数 $ L(Y) (\in \boldsymbol{R}) $ の変数と考える

***
#### Affineレイヤの逆伝播
$$
\frac{\partial L}{\partial X} = \frac{\partial L}{\partial Y} W^T \\ \\
\frac{\partial Y}{\partial W} = X^T \frac{\partial L}{\partial Y} \\ \\
$$

***

#### 導出
- 第 $(l-1)$ 層から第 $l$ 層への変換を考える
- 以下のように定義

$$
\begin{align}
Y &= \left( y_1, y_2, \cdots, y_j, \cdots, y_m \right)
\\ \\
X &= \left( x_1, x_2, \cdots, x_k, \cdots, x_n \right)
\\ \\
W &= \begin{pmatrix}
w_{11} & w_{21} & \cdots & w_{m1} \\
w_{12} & w_{22} & \cdots & w_{m2} \\
\vdots & \vdots & w_{jk} & \vdots \\
w_{1n} & w_{2n} & \cdots & w_{mn}
\end{pmatrix}
\\ \\
B &= \left( b_1, b_2, \cdots, b_j, \cdots, b_m \right)
\end{align}
$$
- $n$: $(l-1)$ 層のノード数
- $m$: $l$ 層のノード数
- $Y$: Affine変換後の $l$ 層への入力．
- $X$: $(l-1)$ 層からの出力．
- $W$: 重み．$w_{jk}$ は $(l-1)$ 層 $k$ 番目のノードから $l$ 層 $j$ 番目のノードへの重み．
- $B$: バイアス．

***

Affine変換を成分表示
$$ Y = \left( x_1, x_2, \cdots, x_k, \cdots, x_n \right)
\begin{pmatrix}
w_{11} & w_{21} & \cdots & w_{m1} \\
w_{12} & w_{22} & \cdots & w_{m2} \\
\vdots & \vdots & w_{jk} & \vdots \\
w_{1n} & w_{2n} & \cdots & w_{mn}
\end{pmatrix}
+ \left( b_1, b_2, \cdots, b_j, \cdots, b_m \right)
$$

$Y$ の成分 $y_j$
$$
\begin{align}
y_j 
&= x_1 w_{j1} + x_2 w_{j2} + \cdots  + x_n w_{jn} + b_j \\ \\
&= \Sigma_{k=1}^{n} x_k w_{jk} + b_j
\end{align}
$$

$Y$ の成分 $y_j$ の偏微分
$$
\begin{align}
\frac{\partial y_j}{\partial x_k} 
&= w_{jk}
\\ \\
\frac{\partial y_j}{\partial w_{jk}} 
&= x_k
\\ \\
\frac{\partial y_j}{\partial b_j} 
&= 1
\end{align}
$$

損失関数の $x_k$ による微分を，Affine変換の合成関数の微分で表す
$$
\begin{align}
\frac{\partial L}{\partial x_k} &= \Sigma_{j=1}^{m} \frac{\partial L}{\partial y_j} \frac{\partial y_j}{\partial x_k}
\\ \\
&= \left( \frac{\partial L}{\partial y_1}, \frac{\partial L}{\partial y_2}, \cdots, \frac{\partial L}{\partial y_m} \right)
\left(
\begin{array}{c}
\frac{\partial y_1}{\partial x_k} \\
\frac{\partial y_2}{\partial x_k} \\
\vdots \\
\frac{\partial y_m}{\partial x_k}
\end{array}
\right)
\\ \\
&= \Sigma_{j=1}^{m} \frac{\partial L}{\partial y_j}
\left(
\begin{array}{c}
w_{1k}\\
w_{2k} \\
\vdots \\
w_{mk}
\end{array}
\right)
\end{align}
$$